# How to generate robustness curves

In [ ]:
from argparse import Namespace
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import foolbox

import provable_robustness_max_linear_regions.data as dt
from provable_robustness_max_linear_regions import models
from provable_robustness_max_linear_regions.models import load_model
from robustness_curves import generate_curve_data

## 1. Load and prepare your dataset
The data needs to be a numpy array. In this case we use 10 datapoints from the CIFAR 10 test set. We also reshape the data to fit our model.

In [ ]:
n_points = 3

_, x_test, _, y_test = dt.get_dataset("cifar10")

x_test = x_test[:n_points]
y_test = y_test[:n_points]

x_test = x_test.reshape(n_points, 1, 32, 32, 3)

## 2. Train and prepare your model
In this case we use a small pretrained convolutional neural network, that was trained with standard training on the CIFAR10 training set and implemented in TensorFlow 1. Most of the steps in this cell might look very different for you, depending on how you implemented your model. Your model does not need to be implemented in TensorFlow 1. Our module supports most of the popular machine learning frameworks, including TensorFlow 2, PyTorch or JAX.

In [ ]:
model_weight_path = "provable_robustness_max_linear_regions/models/plain/2019-02-19 01_20_16 dataset=cifar10 nn_type=cnn_lenet_small p_norm=inf lmbd=0.0 gamma_rb=0.0 gamma_db=0.0 ae_frac=0.0 epoch=100.mat"

model_args = Namespace()
n_test_ex, one, model_args.height, model_args.width, model_args.n_col = x_test.shape
model_args.n_in, model_args.n_out = model_args.height * model_args.width * model_args.n_col, y_test.shape[1]
model_args.n_hs = []
model_args.seed = 1
model_args.nn_type = "cnn"
model_args.dataset = "cifar10"

sess = tf.InteractiveSession()
model, _input, _logits, _ = load_model(sess, model_args, model_weight_path)

## 3. Wrap the model with a foolbox model wrapper
Use the correct wrapper for the machine learning framework, in which you implemented your model. The list of possible model wrappers can be found in the documentation of foolbox 2.4 (https://foolbox.readthedocs.io/en/v2.4.0/modules/models.html).

In [ ]:
f_model = foolbox.models.TensorFlowModel(_input, _logits, (0,1))      

## 4. Define the arguments for the module
The norms are the $\ell_p$ norms for which to calculate the robustness curves. We support $\ell_\infty$, $\ell_1$ and $\ell_2$.

In [ ]:
args = Namespace()

args.inputs = x_test
args.labels = y_test
args.f_model = f_model
args.norms = ["inf", "2"]
args.save = True
args.plot = True

## 5. Generate and plot the curves

In [ ]:
robustness_curve_data = generate_curve_data(args)

In [ ]:
tf.reset_default_graph()
sess.close()